In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import dagshub
import os
import mlflow.sklearn
from sklearn.pipeline import Pipeline
import mlflow
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv("../data/external/emotion_dataset.csv")
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [4]:
df.drop(columns=["tweet_id"], inplace=True)
df = df[df["sentiment"].isin(["happiness", "sadness"])]

In [5]:
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=test_size, random_state=42)

In [6]:
encode = LabelEncoder()
y_train = encode.fit_transform(y_train)
y_test = encode.transform(y_test)

In [7]:
def lemmatization(text: str) -> str:
    lemmatizer = WordNetLemmatizer()

    text = text.split()
    text=[lemmatizer.lemmatize(y) for y in text]

    return " ".join(text)

def remove_stop_words(text: str) -> str:
    try:
        stop_words = set(stopwords.words("english"))
    except Exception:
        print("An error has occurred. If stopwords aren't there please download.")
        raise
    else:
        text=[i for i in str(text).split() if i not in stop_words]
        return " ".join(text)

def removing_numbers(text: str) -> str:
    text = "".join([i for i in text if not i.isdigit()])
    return text

def lower_case(text: str) -> str:
    text = text.split()

    text=[y.lower() for y in text]

    return " ".join(text)

def removing_punctuations(text: str) -> str:
    ## Remove Punctuations
    text = re.sub("[%s]" % re.escape("""!"#$%&'()*+,.-./:;<=>?@[\]^_`{|}~"""), ' ', text)
    text = text.replace(':', "")

    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text = " ".join(text.split())
    return text.strip()

def removing_urls(text: str) -> str:
    url_pattern = re.compile(r"https?://\S+|www\.\S+")
    return url_pattern.sub(r"", text)

def remove_small_sentences(df: pd.DataFrame) -> None:
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan

def normalize_text(df: pd.DataFrame) -> pd.DataFrame:
    df.content = df.content.apply(lambda content : lower_case(content))
    df.content = df.content.apply(lambda content : remove_stop_words(content))
    df.content = df.content.apply(lambda content : removing_numbers(content))
    df.content = df.content.apply(lambda content : removing_punctuations(content))
    df.content = df.content.apply(lambda content : removing_urls(content))
    df.content = df.content.apply(lambda content : lemmatization(content))
    return df

<>:32: SyntaxWarning: invalid escape sequence '\]'
<>:36: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\]'
<>:36: SyntaxWarning: invalid escape sequence '\s'
C:\Users\RR\AppData\Local\Temp\ipykernel_2244\3701265362.py:32: SyntaxWarning: invalid escape sequence '\]'
  text = re.sub("[%s]" % re.escape("""!"#$%&'()*+,.-./:;<=>?@[\]^_`{|}~"""), ' ', text)
C:\Users\RR\AppData\Local\Temp\ipykernel_2244\3701265362.py:36: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)


In [8]:
X_train = normalize_text(X_train)
X_test = normalize_text(X_test)

In [9]:
max_cols = 100
bag_of_words = CountVectorizer(max_features=max_cols)
logistic_regression = LogisticRegression()

In [10]:
train_X = X_train["content"].values

In [11]:
pipe = Pipeline([
    ("bag_of_words", bag_of_words),
    ("model", logistic_regression)
])

In [12]:
pipe.fit(X_train["content"].values, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('bag_of_words', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (strip_accents and lowercase) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [13]:
y_pred = pipe.predict(X_test["content"].values)

In [14]:
accuracy = accuracy_score(y_pred, y_test)
precision = precision_score(y_pred, y_test)
recall = recall_score(y_pred, y_test)
f1_score = f1_score(y_pred, y_test)
accuracy, precision, recall, f1_score

(0.7113253012048193,
 0.7726415094339623,
 0.6958368734069669,
 0.7322306660706303)

In [15]:
params = {
    "bag_of_words__max_features": [50, 80, 100, 150, 200, 300],
    "model__C": [0.01, 0.1, 1, 10],
    "model__max_iter": [150, 200, 250, 300, 500]
}

In [16]:
grid_search = GridSearchCV(pipe, params, scoring="f1", n_jobs=-1,cv=5, verbose=2)

In [17]:
grid_search.fit(train_X, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...egression())])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'bag_of_words__max_features': [50, 80, ...], 'model__C': [0.01, 0.1, ...], 'model__max_iter': [150, 200, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'f1'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate i

In [18]:
mlflow.set_tracking_uri("https://dagshub.com/PriyanshuMewal/mini-project.mlflow")

dagshub.init(repo_owner='PriyanshuMewal', repo_name='mini-project', mlflow=True)

mlflow.set_experiment("Exp3: Logistic Regression with Bag of Words")

with mlflow.start_run(run_name="grid_search") as parent:

    for i in range(len(grid_search.cv_results_["params"])):
    
        with mlflow.start_run(nested=True) as children:
            # log params
            mlflow.log_params(grid_search.cv_results_["params"][i])
            mlflow.log_param("test_size", test_size)
            mlflow.log_param("vectorizer", "Bag Of Words")
            mlflow.log_param("model", "Logistic Regression")
            
            # log metrics
            mlflow.log_metric('f1_score', grid_search.cv_results_["mean_test_score"][i])

    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    # log best params:
    mlflow.log_params(best_params)
    mlflow.log_param("test_size", test_size)
    mlflow.log_param("vectorizer", "Bag Of Words")
    mlflow.log_param("model", "Logistic Regression")
    
    # log metrics
    mlflow.log_metric('f1_score', best_score)

Accessing as PriyanshuMewal

Initialized MLflow to track repo "PriyanshuMewal/mini-project"

Repository PriyanshuMewal/mini-project initialized!

2025/12/17 05:40:54 INFO mlflow.tracking.fluent: Experiment with name 'Exp3: Logistic Regression with Bag of Words' does not exist. Creating a new experiment.


🏃 View run funny-snail-427 at: https://dagshub.com/PriyanshuMewal/mini-project.mlflow/#/experiments/4/runs/5d1080ebbf7e4179ba67737d4154d81c
🧪 View experiment at: https://dagshub.com/PriyanshuMewal/mini-project.mlflow/#/experiments/4
🏃 View run thoughtful-robin-392 at: https://dagshub.com/PriyanshuMewal/mini-project.mlflow/#/experiments/4/runs/f8559b50303c464e909577167001357c
🧪 View experiment at: https://dagshub.com/PriyanshuMewal/mini-project.mlflow/#/experiments/4
🏃 View run clumsy-conch-327 at: https://dagshub.com/PriyanshuMewal/mini-project.mlflow/#/experiments/4/runs/64da54b27fd241f8a9f6a266f2aad774
🧪 View experiment at: https://dagshub.com/PriyanshuMewal/mini-project.mlflow/#/experiments/4
🏃 View run orderly-bug-514 at: https://dagshub.com/PriyanshuMewal/mini-project.mlflow/#/experiments/4/runs/0f42a64e679d48e989471343d6e2c59a
🧪 View experiment at: https://dagshub.com/PriyanshuMewal/mini-project.mlflow/#/experiments/4
🏃 View run kindly-gnu-598 at: https://dagshub.com/PriyanshuMew

AttributeError: 'GridSearchCV' object has no attribute 'best_params'

In [20]:
df = pd.DataFrame(grid_search.cv_results_)[["param_bag_of_words__max_features","param_model__C","param_model__max_iter", "mean_test_score", "params"]]
df[df["mean_test_score"] >= 0.75]

,param_bag_of_words__max_features,param_model__C,param_model__max_iter,mean_test_score,params
105,300,0.1,150,0.754377,"{'bag_of_words__max_features': 300, 'model__C'..."
106,300,0.1,200,0.754377,"{'bag_of_words__max_features': 300, 'model__C'..."
107,300,0.1,250,0.754377,"{'bag_of_words__max_features': 300, 'model__C'..."
108,300,0.1,300,0.754377,"{'bag_of_words__max_features': 300, 'model__C'..."
109,300,0.1,500,0.754377,"{'bag_of_words__max_features': 300, 'model__C'..."
110,300,1.0,150,0.750103,"{'bag_of_words__max_features': 300, 'model__C'..."
111,300,1.0,200,0.750103,"{'bag_of_words__max_features': 300, 'model__C'..."
112,300,1.0,250,0.750103,"{'bag_of_words__max_features': 300, 'model__C'..."
113,300,1.0,300,0.750103,"{'bag_of_words__max_features': 300, 'model__C'..."
114,300,1.0,500,0.750103,"{'bag_of_words__max_features': 300, 'model__C'..."
